In [1]:
import numpy as np

from experiments.utils.SimpleComparison import SimpleComparison
from hdimvis.algorithms.spring_force_algos.chalmers96_algo.Chalmers96 import Chalmers96
from hdimvis.visualise_layouts_and_metrics.plot import show_layout,show_generation_metrics
from hdimvis.algorithms.stochastic_ntet_algo.SNeD import SNeD
from hdimvis.create_low_d_layout.LayoutCreation import LayoutCreation
from hdimvis.data_fetchers.DataFetcher import DataFetcher
from hdimvis.visualise_layouts_and_metrics.plot import show_layout
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from experiments.utils.get_kmeans_f1 import get_kmeans_f1
from experiments.utils.get_f1_for_best_k_with_knn import get_f1_for_best_k_with_knn
from experiments.utils.get_avg_classwise_f1 import get_avg_classwise_f1
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
from pathlib import Path
from definitions import PROJECT_ROOT
import pickle
from scipy import stats
import matplotlib.pyplot as plt
from time import perf_counter
from  sklearn.manifold import SpectralEmbedding, MDS
from hdimvis.metrics.distance_measures.euclidian_and_manhattan import euclidean

In [2]:
output_dir= (Path(PROJECT_ROOT).joinpath(
    Path(f"experiments/knnd/out/"))).resolve().absolute()

path_to_pickle = (Path(output_dir).joinpath(Path(f"layouts.pickle"))).resolve()
with open(path_to_pickle, 'rb') as file:
    layouts = pickle.load(file)
print(layouts)

EOFError: Ran out of input

In [ ]:
for name, dict in layouts.items():
    print(name)
    for type, ls in dict.items():
        print(type)
        for l in ls:
            show_layout(l, use_labels=True)


In [3]:
output_dir= (Path(PROJECT_ROOT).joinpath(
    Path(f"experiments/knnd/out/"))).resolve().absolute()

path_to_pickle = (Path(output_dir).joinpath(Path(f"times.pickle"))).resolve()
with open(path_to_pickle, 'rb') as file:
    times = pickle.load(file)
print(times)

EOFError: Ran out of input

In [ ]:
# rna N3k -  knnd first removed - outlier


names = ['rna N3k','coli20', 'globe', 'fashion mnist', 'mnist']


for name in names:
    if name == 'rna N3k':
        knnd = np.array(times[name]['knnd'])[1:]  # remove the outlier for rna n3k
    else:
        knnd = np.array(times[name]['knnd'])

    no_knnd = np.array(times[name]['no knnd'])

    print(f"{name}:"
      f"Mean kNNd: {np.mean(knnd)}, SD: {np.std(knnd)} \n"
      f"Mean no knnd {np.mean(no_knnd)}, SD: {np.std(no_knnd)} \n")
    # tstat, pvalue = stats.ttest_ind(knnd, no_knnd[:,1], equal_var=False, alternative='greater')
    # print(pvalue)
    print('#'*20)


In [ ]:
# see what good bounds for k might be for knn classifier
for dataset_name, version_dict in layouts:
    print(dataset_name)
    for j, version in enumerate(['knnd', 'no knnd']):
        print(version)
        for i,layout in enumerate(version_dict[version]):
            print(i)
            get_f1_for_best_k_with_knn(lower_bound=1, upper_bound=200,
                                         cross_validation_folds=10,
                                         data=layout.data,
                                         labels=layout.labels,
                                       show_averages=True)



In [ ]:
results_arr = np.zeros((5,2,20,3)) # 5 datasets, 2 version - knnd and not, 20 repeats, 3 stats

for dataset_name, version_dict in layouts:
    for j, version in enumerate(['knnd', 'no knnd']):
        for i,layout in enumerate(version_dict[version]):
            stress = layout.get_final_stress()
            kmeans_f1 = get_kmeans_f1(layout.get_final_positions(), layout.labels)
            f1_for_best_k_with_knn =  get_f1_for_best_k_with_knn(lower_bound=1, upper_bound=3,
                                                                 cross_validation_folds=10,
                                                                 data=layout.data,
                                                                 labels=layout.labels)

            results_arr[names.index(dataset_name),j,i,0] = stress
            results_arr[names.index(dataset_name),j,i,1] = kmeans_f1
            results_arr[names.index(dataset_name),j,i,2] = f1_for_best_k_with_knn


# see the means
for i in range(3):
    means_arr = np.mean(results_arr[:,:,:,i], axis=2)


# _, pvalues_stress = stats.ttest_ind(results_arr[:,0,:,0], results_arr[:,1,:,0],
#                                     axis=1,equal_var=False,alternative='two sided' )
# _, pvalues_kmeans = stats.ttest_ind(results_arr[:,0,:,1], results_arr[:,1,:,1],
#                                     axis=1,equal_var=False,alternative='two sided' )
# _, pvalues_knn = stats.ttest_ind(results_arr[:,0,:,2], results_arr[:,1,:,2],
#                                     axis=1,equal_var=False,alternative='two sided' )